<a href="https://colab.research.google.com/github/naveenkumark1/Extensive_Vision_and_AI_V.4/blob/master/Week4/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [0]:
'''class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))

        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        
        x = self.conv7(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)'''


class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Sequential(
                                nn.Conv2d(1,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.05),
                                nn.Conv2d(16,32,3),nn.ReLU(),nn.BatchNorm2d(32),nn.Dropout2d(0.05),
                                nn.Conv2d(32,10,3),nn.ReLU(),#nn.BatchNorm2d(128),nn.Dropout2d(0.05),
                                nn.MaxPool2d(2,2)
                                )

    self.conv2 = nn.Sequential(
                                nn.Conv2d(10,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.05),
                                nn.Conv2d(16,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.05),
                                nn.Conv2d(16,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.05),
                                #nn.MaxPool2d(2,2)
                                )
    
    self.conv3 = nn.Sequential(
                                nn.Conv2d(16,16,3),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout2d(0.05),
                                nn.Conv2d(16,10,3),nn.ReLU(),#nn.BatchNorm2d(10),nn.Dropout2d(0.05),
                                
                                )

## Batchnorm () - Comment

  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)

    x = x.view(-1,10)

    return F.log_softmax(x)

In [50]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 22, 22]           2,890
             ReLU-10           [-1, 10, 22, 22]               0
        MaxPool2d-11           [-1, 10, 11, 11]               0
           Conv2d-12             [-1, 16, 9, 9]           1,456
             ReLU-13             [-1, 16, 9, 9]               0
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [53]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0561, Accuracy: 9821/10000 (98.21%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0369, Accuracy: 9880/10000 (98.8%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0229, Accuracy: 9931/10000 (99.31%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0212, Accuracy: 9928/10000 (99.28%)



loss=0.0007333506946451962 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.22it/s]



Test set: Average loss: 0.0226, Accuracy: 9925/10000 (99.25%)

